# Zero-Shot 프롬프트를 사용하여 텍스트 생성을 위한 베드락 모델 호출

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

이 노트북에서는 LLM을 사용하여 지원 엔지니어로부터 받은 고객 서비스 품질에 대해 부정적인 피드백을 제공한 고객에게 이메일 응답을 생성하는 방법을 보여드리겠습니다. 

여기서는 Boto3 API를 사용하는 Bedrock의 Amazon Titan Text 대형 모델을 사용합니다. 

이 예제에서 사용된 프롬프트는 제로 샷 프롬프트라고 하는데, 프롬프트 외에 분류와 함께 텍스트의 예를 제공하지 않기 때문입니다.

**참고:** *이 노트북은 AWS 환경 내부 또는 외부에서 실행할 수 있습니다.*

#### Context
Amazon Bedrock의 텍스트 생성 기능을 시연하기 위해 Boto3 클라이언트를 사용하여 Amazon Bedrock API와 통신하는 방법을 살펴보겠습니다. 사용 가능한 다양한 구성과 간단한 입력이 어떻게 원하는 출력으로 이어질 수 있는지 보여드리겠습니다.

#### Pattern
추가 예제를 제공하지 않고 작업, 명령어 및 내부 모델에 대한 입력으로 구성된 입력을 Amazon Bedrock API에 제공하여 출력을 생성하기만 하면 됩니다. 여기서는 강력한 LLM이 당면한 작업을 쉽게 이해하고 매력적인 결과물을 생성하는 방법을 보여드리기 위한 것입니다.

![](./images/bedrock.jpg)

#### Use case
Amazon Bedrock에서 모델의 생성 기능을 보여주기 위해 이메일 생성의 사용 사례를 살펴보겠습니다.

#### Persona
당신은 애니컴퍼니의 고객 서비스 매니저인 밥이고, 일부 고객이 고객 서비스에 만족하지 못하고 고객 지원 엔지니어가 제공하는 서비스에 대해 부정적인 피드백을 제공하고 있습니다. 이제 이러한 고객들에게 서비스 불만족에 대해 겸손하게 사과하고 신뢰를 회복하고 싶습니다. 이전 이메일 서신에서 얻은 고객의 정서에 맞춰 인간적이고 개인화된 이메일을 대량으로 생성하려면 LLM의 도움이 필요합니다.

#### Implementation
이 사용 사례를 충족하기 위해 이 노트북에서는 고객의 이전 이메일을 기반으로 감사 메모가 포함된 이메일을 생성하는 방법을 보여드리며, Boto3 클라이언트와 함께 Amazon Bedrock API를 사용하는 Amazon Titan Text Large 모델을 사용하겠습니다.

## Setup

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 필요한 라이브러리가 설치되어 있는지 확인하고 베드락에 연결해야 합니다.

설정 작동 방식과 ⚠️ **변경이 필요한지 여부**에 대한 자세한 내용은 [Bedrock 기본환경 설정 노트북](../00_Setup/setup.ipynb) 노트북을 참조하세요.

In [2]:
! pip list | grep langchain
! pip list | grep opensearch

langchain                            0.0.249
opensearch-py                        2.3.2


In [3]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import print_ww

In [4]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


## Generate text

위에서 설명한 사용 사례에 따라 Amazon Bedrock 서비스에서 이메일을 생성하기 위한 입력을 준비해 보겠습니다.

In [5]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""


Amazon Titan 대형 모델부터 사용해보겠습니다. Amazon Titan Large는 최대 4k 토큰의 컨텍스트 창을 지원하며 다음 매개 변수를 허용합니다:
- `inputText`: LLM에 대한 프롬프트
- `textGenerationConfig`: 이는 모델이 출력을 생성하는 동안 고려할 파라미터입니다.

In [6]:
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":4096,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

아마존 베드락 API는 다음을 허용하는 API `invoke_model`을 제공합니다:
- `modelId`: 아마존 베드락에서 사용할 수 있는 다양한 파운데이션 모델에 대한 ARN 모델입니다.
- `accept`: 입력 요청 유형
- `contentType`: 출력의 콘텐츠 유형
- `body`: 프롬프트와 구성으로 구성된 json 문자열입니다.

Amazon Bedrock에서 사용 가능한 텍스트 생성 모델의 ID는 다음과 같습니다:
- `amazon.titan-tg1-large`
- `ai21.j2-grande-instruct`
- `ai21.j2-jumbo-instruct`
- `anthropic.claude-instant-v1`
- `anthropic.claude-v1`

#### Amazon Titan 대형 언어 모델(Amazon Titan LLM) 호출

먼저, 앞서 생성한 프롬프트를 기반으로 모델이 출력을 생성하는 방법을 살펴봅니다.

##### Complete Output Generation

In [7]:
modelId = 'amazon.titan-tg1-large' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

outputText = response_body.get('results')[0].get('outputText')


In [8]:
# 응답의 관련 부분은 첫 번째 줄 바꿈 문자 이후부터 시작됩니다.
# 아래에서는 '\n'이 처음 나오는 부분부터 응답을 출력합니다.

email = outputText[outputText.index('\n')+1:]
print_ww(colored(email, "green"))


Subject: Apology for Poor Service Experience

Dear John Doe,

I hope this email finds you well. I am writing to express my sincere apologies for the poor service
experience you had when you contacted our customer support engineer Jane Doe.

We strive to provide excellent service to all our customers, and it is disheartening to hear that we
fell short of your expectations. We take feedback like yours very seriously, and we are committed to
resolving the issue and ensuring that we provide better service in the future.

If you would like to discuss this matter further, please do not hesitate to contact me directly. We
value your opinion and want to make things right for you.

Once again, I apologize for any inconvenience caused.

Best regards,

Bob Customer Service Manager


##### 스트리밍 출력 생성
위는 입력 요청을 이해하고 다양한 양식에 대한 고유한 이해를 사용하여 Amazon Titan Large 모델이 생성한 이메일의 예입니다. API에 대한 이 요청은 동기식이며 모델에서 전체 출력이 생성될 때까지 기다립니다.

또한 베드락은 모델에서 청크 형태로 생성되는 출력을 스트리밍할 수 있도록 지원합니다. 아래는 스트리밍 옵션으로 모델을 호출하는 예제입니다. invoke_model_with_response_stream은 읽을 수 있는 `ResponseStream`을 반환합니다.

In [9]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            output.append(text)
            print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
            i+=1

		**Chunk 1**
 "Jane Doe"
Subject: Apology for Poor Service Experience

Dear John Doe,

I hope this email finds you well. I am writing to express my sincere apologies for the poor service experience you had when

		**Chunk 2**
 you contacted our customer support engineer Jane Doe.

We strive to provide excellent service to all our customers, and it is disheartening to hear that we fell short of your expectations. We take feedback like yours very seriously, and we are committed to resolving the issue and ensuring that we provide better service in the future.

If

		**Chunk 3**
 you would like to discuss this matter further, please do not hesitate to contact me directly. We value your opinion and want to make things right for you.

Once again, I apologize for any inconvenience caused.

Best regards,

Bob Customer Service Manager



위의 방법은 모델의 출력을 빠르게 가져와서 사용자가 읽는 대로 서비스가 완료하도록 하는 데 도움이 됩니다. 이는 모델에 생성을 요청하는 긴 텍스트 조각이 있는 사용 사례에 유용합니다. 나중에 생성된 모든 청크를 결합하여 완전한 출력을 형성하고 사용 사례에 사용할 수 있습니다.

In [13]:
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(colored(complete_output, "green"))

		**COMPLETE OUTPUT**

 "Jane Doe"
Subject: Apology for Poor Service Experience

Dear John Doe,

I hope this email finds you well. I am writing to express my sincere apologies for the poor service experience you had when you contacted our customer support engineer Jane Doe.

We strive to provide excellent service to all our customers, and it is disheartening to hear that we fell short of your expectations. We take feedback like yours very seriously, and we are committed to resolving the issue and ensuring that we provide better service in the future.

If you would like to discuss this matter further, please do not hesitate to contact me directly. We value your opinion and want to make things right for you.

Once again, I apologize for any inconvenience caused.

Best regards,

Bob Customer Service Manager


## Conclusion
이제 아마존 베드락 API에 `boto3` SDK를 사용하여 실험해 보았습니다. 이 API를 사용하여 고객의 부정적인 피드백으로 인해 고객에게 응답하는 이메일을 생성하는 사용 사례를 확인 해봤습니다.

### Take aways
- 이 노트북을 개조하여 Anthropic Claude 및 AI21 Labs Jurassic 모델과 같은 Amazon Bedrock을 통해 제공되는 다양한 모델을 실험해 보세요.
- 프롬프트를 특정 사용 사례에 맞게 변경하고 다양한 모델의 출력을 평가하세요.
- 토큰 길이를 변경하여 서비스의 지연 시간과 응답성을 파악하세요.
- 다양한 프롬프트 엔지니어링 원칙을 적용하여 더 나은 결과를 얻으세요.

## Thank You